In [1]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import itertools

Using TensorFlow backend.


In [2]:
# all images will be converted to this size
ROWS = 256
COLS = 256
CHANNELS = 3

In [3]:
train_image_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255, rotation_range=45)
test_image_generator = ImageDataGenerator(horizontal_flip=False, rescale=1./255, rotation_range=0)

train_generator = train_image_generator.flow_from_directory('train', target_size=(ROWS, COLS), class_mode='categorical')
test_generator = test_image_generator.flow_from_directory('test', target_size=(ROWS, COLS), class_mode='categorical')

Found 5994 images belonging to 200 classes.
Found 5794 images belonging to 200 classes.


In [12]:
train_generator.reset()
test_generator.reset()

model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=(ROWS, COLS, CHANNELS)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(400))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 254, 254, 64)      1792      
_________________________________________________________________
activation_28 (Activation)   (None, 254, 254, 64)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 252, 252, 64)      36928     
_________________________________________________________________
activation_29 (Activation)   (None, 252, 252, 64)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 61, 61, 64)        36928     
_________________________________________________________________
activation_30 (Activation)   (None, 61, 61, 64)        0         
__________

In [15]:
tensorboard = TensorBoard(log_dir='./logs/custom')

model.fit_generator(train_generator, steps_per_epoch=512, epochs=10, callbacks=[tensorboard], verbose=2)

Epoch 1/10
 - 434s - loss: 4.4682 - acc: 0.0687
Epoch 2/10
 - 440s - loss: 4.1851 - acc: 0.0919
Epoch 3/10
 - 443s - loss: 3.9278 - acc: 0.1270
Epoch 4/10
 - 428s - loss: 3.6948 - acc: 0.1615
Epoch 5/10
 - 437s - loss: 3.4944 - acc: 0.1935
Epoch 6/10
 - 439s - loss: 3.3103 - acc: 0.2196
Epoch 7/10
 - 438s - loss: 3.1253 - acc: 0.2492
Epoch 8/10
 - 443s - loss: 2.9927 - acc: 0.2757
Epoch 9/10
 - 431s - loss: 2.8474 - acc: 0.2998
Epoch 10/10
 - 430s - loss: 2.7354 - acc: 0.3271


In [16]:
print(model.evaluate_generator(test_generator, steps=1000))

[3.3455331521880121, 0.22266875981161696]
